In [10]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize

In [4]:
CORPUS = pd.read_csv('/Users/ishathukral/Downloads/DS5001project/output/CORPUS.csv', sep='|')

In [5]:
BOW = CORPUS.groupby(['doc_id', 'term_str']).size().reset_index(name='n')

doc_freq = CORPUS.groupby('term_str')['doc_id'].nunique().reset_index()
doc_freq.columns = ['term_str', 'document_frequency'] 

BOW = BOW.merge(doc_freq, on='term_str', how='left')

N_docs = CORPUS['doc_id'].nunique()

BOW['idf'] = np.log((N_docs / BOW['document_frequency']) + 1)

BOW['tf'] = BOW['n'] / BOW.groupby('doc_id')['n'].transform('sum')

BOW['tfidf'] = BOW['tf'] * BOW['idf']

BOW.head()

,doc_id,term_str,n,document_frequency,idf,tf,tfidf
0,1,!,19,120,1.005522,0.003689,0.003710
1,1,(,1,9,3.182673,0.000194,0.000618
2,1,),1,9,3.182673,0.000194,0.000618
3,1,",",367,208,0.693147,0.071262,0.049395
4,1,.,350,208,0.693147,0.067961,0.047107


In [19]:
BOW.to_csv('/Users/ishathukral/Downloads/DS5001project/output/BOW.csv', index=False, sep='|')

In [7]:


data = BOW['n']  
row_indices = BOW['doc_id'] 
col_indices = pd.Categorical(BOW['term_str'], categories=BOW['term_str'].unique()).codes

DTM = csr_matrix((data, (row_indices, col_indices)), shape=(BOW['doc_id'].max() + 1, len(BOW['term_str'].unique())))
dtm_df = pd.DataFrame(DTM.toarray(), columns=BOW['term_str'].unique())

dtm_df


,!,(,),",",.,1,10,"100,000",11,12,...,seehutee,shankar,shi-aamaa,sindhavee,soohau,surmaanand,sweet-,taylangee,thrilling,voiced
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19,1,1,367,350,4,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,99,70,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,21,9,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,82,71,8,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,1,0,0,259,190,34,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
205,2,0,0,246,205,35,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
206,3,0,0,862,804,113,2,0,2,2,...,0,0,0,0,0,0,0,0,0,0
207,76,1,1,3827,2791,50,20,0,17,15,...,0,0,0,0,0,0,0,0,0,0


In [16]:

dtm_df.to_csv('/Users/ishathukral/Downloads/DS5001project/output/DTM.csv', index=False, sep='|')

In [9]:

data = BOW['tfidf']
row_indices = BOW['doc_id'] - 1 
col_indices = pd.Categorical(BOW['term_str'], categories=BOW['term_str'].unique()).codes

# sparse matrix
TFIDF_DTM = csr_matrix((data, (row_indices, col_indices)), shape=(BOW['doc_id'].max(), len(BOW['term_str'].unique())))
tfidf_dtm_df = pd.DataFrame(TFIDF_DTM.toarray(), columns=pd.Categorical(BOW['term_str'], categories=BOW['term_str'].unique()).categories)

tfidf_dtm_df


,!,(,),",",.,1,10,"100,000",11,12,...,seehutee,shankar,shi-aamaa,sindhavee,soohau,surmaanand,sweet-,taylangee,thrilling,voiced
0,0.003710,0.000618,0.000618,0.049395,0.047107,0.000538,0.000237,0.000771,0.000250,0.000258,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.054118,0.038265,0.003827,0.000963,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.064983,0.027850,0.006189,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.005617,0.000000,0.000000,0.052922,0.045823,0.005163,0.000000,0.000000,0.001201,0.001236,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.004953,0.000000,0.000000,0.030731,0.030731,0.010244,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.000253,0.000000,0.000000,0.045118,0.033098,0.005923,0.000307,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
204,0.000503,0.000000,0.000000,0.042661,0.035550,0.006070,0.000305,0.000000,0.000323,0.000332,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
205,0.000219,0.000000,0.000000,0.043369,0.040451,0.005685,0.000177,0.000000,0.000187,0.000193,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
206,0.001405,0.000059,0.000059,0.048774,0.035571,0.000637,0.000449,0.000000,0.000403,0.000366,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
tfidf_dtm_df.to_csv('/Users/ishathukral/Downloads/DS5001project/output/TFIDF_DTM.csv', index=False, sep='|')

In [14]:
normalized_dtm = normalize(TFIDF_DTM, norm='l2', axis=1)

normalized_dtm_df = pd.DataFrame(normalized_dtm.toarray(), columns=tfidf_dtm_df.columns)

normalized_dtm_df

,!,(,),",",.,1,10,"100,000",11,12,...,seehutee,shankar,shi-aamaa,sindhavee,soohau,surmaanand,sweet-,taylangee,thrilling,voiced
0,0.035495,0.005913,0.005913,0.472616,0.450723,0.005151,0.002269,0.007376,0.002397,0.002467,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.489713,0.346262,0.034626,0.008714,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.470699,0.201728,0.044828,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.050633,0.000000,0.000000,0.477016,0.413026,0.046538,0.000000,0.000000,0.010827,0.011145,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.038654,0.000000,0.000000,0.239813,0.239813,0.079938,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.002271,0.000000,0.000000,0.405435,0.297423,0.053223,0.002758,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
204,0.004771,0.000000,0.000000,0.404491,0.337076,0.057550,0.002897,0.000000,0.003060,0.003150,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
205,0.002059,0.000000,0.000000,0.407824,0.380384,0.053462,0.001667,0.000000,0.001761,0.001813,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
206,0.013994,0.000583,0.000583,0.485744,0.354249,0.006346,0.004472,0.000000,0.004016,0.003648,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:

normalized_dtm_df.to_csv('/Users/ishathukral/Downloads/DS5001project/output/Normalized_TFIDF_L2_DTM.csv', index=False, sep='|')
